In [1]:
import sys
import string
import simplejson
from twython import Twython, TwythonError
import datetime
import time
import pickle

import numpy as np
import pandas as pd



In [2]:
t = Twython(app_key='', 
    app_secret='',
    oauth_token='',
    oauth_token_secret='')



In [52]:
#upload the userinfo pickle file
homeusers = pickle.load( open( "userLoc_GaussianMixture.p", "rb" ) )


In [17]:
userdf = pd.DataFrame.from_dict(homeusers, 'index')

In [22]:
userdf.head(2)

,0
lancewhite,POINT (-73.86230676 40.76844627)
julietterichey,POINT (-73.968379035 40.679262225)


In [44]:
#get the usernames from the index and put them in a list
names = userdf.index
# for testing - use only a subset of names list
names = names[:5]
names = names.tolist()

In [45]:
names

['lancewhite', 'julietterichey', 'LaFemFatale', 'danreilly11', 'thejaymfd']

In [46]:
#use tywthon to get userinfo and put in a df
usersinfo = pd.DataFrame(t.lookup_user(screen_name = names))

In [57]:
#test 
#get timeline
#get starting twitter id from lancewhite (first name on list)
timeline = pd.DataFrame(t.get_user_timeline(screen_name="lancewhite", count="5"))
maxid=timeline.id.max()
maxid

836709571121020928

In [58]:
#cycle through all the ids from the user list

for i in range(0,len(names)):
    #get the most recent tweet from this user
    timeline = pd.DataFrame(t.get_user_timeline(screen_name="lancewhite", count="2"))
    maxid=timeline.id.max()
    lid=[maxid]
    #call this user 16 times, each time pulling 200 tweets (including retweets)
    for call in range(0, 16):# lid[-1] pulls the last item in the list
        nextset = t.get_user_timeline(screen_name=names[i], count="200",max_id=lid[-1])
        if 'coordinates' in nextset:
            nextset['coordinates'].append(nextset['attributes']['bounding_box']['coordinates'])
        df = pd.DataFrame(nextset)
        usertweets=pd.concat([df,usertweets])
        #append the last id to
        lid.append(usertweets["id"].min())
    time.sleep(100)

In [59]:
#check number of unique tweet ids
print usertweets.id.nunique()
#Most recent tweet:
usertweets.id.max()
#verify
usertweets[['coordinates', 'place']].tail(5)
#get column names
usertweets.columns

2950


Index([u'contributors', u'coordinates', u'created_at', u'entities',
       u'extended_entities', u'favorite_count', u'favorited', u'geo', u'id',
       u'id_str', u'in_reply_to_screen_name', u'in_reply_to_status_id',
       u'in_reply_to_status_id_str', u'in_reply_to_user_id',
       u'in_reply_to_user_id_str', u'is_quote_status', u'lang', u'place',
       u'possibly_sensitive', u'quoted_status', u'quoted_status_id',
       u'quoted_status_id_str', u'retweet_count', u'retweeted',
       u'retweeted_status', u'source', u'text', u'truncated', u'user'],
      dtype='object')

In [60]:
usertweets.tail(2)

,contributors,coordinates,created_at,entities,extended_entities,favorite_count,favorited,geo,id,id_str,...,quoted_status,quoted_status_id,quoted_status_id_str,retweet_count,retweeted,retweeted_status,source,text,truncated,user
197,None,None,Sun Dec 18 15:47:14 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 14...",NaN,0,False,None,810511752081395712,810511752081395712,...,NaN,NaN,NaN,0,False,NaN,"<a href=""https://untappd.com"" rel=""nofollow"">U...",I just earned the 'Wheel of Styles (Level 10)'...,False,"{u'follow_request_sent': False, u'has_extended..."
198,None,None,Sun Dec 18 15:47:11 +0000 2016,"{u'symbols': [], u'user_mentions': [{u'id': 14...",NaN,0,False,None,810511737132875776,810511737132875776,...,NaN,NaN,NaN,0,False,NaN,"<a href=""https://untappd.com"" rel=""nofollow"">U...",I just earned the 'Heavy Weight (Level 13)' b...,False,"{u'follow_request_sent': False, u'has_extended..."


In [70]:
#push results to pickle file


In [71]:
#plot lat to long
